# 📚 Klassieke Machine‑Learning Demo: CIFAR‑10 met SVM

In dit project laten we zien hoe je **CIFAR‑10** – een beeld­dataset – toch met een **klassiek ML‑model** (Support Vector Machine) kunt aanpakken, zónder deep learning.

> **Belangrijk verschil** ↔ Bij een CNN (Deep Learning model) leert het netwerk zelf visuele patronen. Met een klassiek ML‑model moeten we de afbeeldingen **flattenen** tot tabellen en (optioneel) de dimensie verkleinen met PCA. Dat is minder krachtig, maar je kan alsnog de klassieke ML‐principes toepassen.

## 📦 Stap 1 – Dataset importeren & verkennen
We halen CIFAR‑10 binnen via Keras. Daarna printen we de vorm en bekijken we enkele voorbeelden.

In [ ]:
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('Train shape:', X_train.shape)
print('Test shape:', X_test.shape)

class_names = ['vliegtuig','auto','vogel','kat','hert','hond','kikker','paard','schip','vrachtwagen']

# Voorbeeldbeelden tonen
plt.figure(figsize=(10,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(X_train[i])
    plt.title(class_names[int(y_train[i])])
    plt.axis('off')
plt.tight_layout()
plt.show()

## 🔍 Wat gebeurt hier?
We laden CIFAR-10 in via Keras. Het bevat 60.000 kleurafbeeldingen van 32×32 pixels, verdeeld over 10 klassen (zoals hond, kat, vliegtuig, auto).

Keras splitst het alvast in een trainingsset (50.000) en een testset (10.000).

Elke afbeelding heeft 3 kleurkanalen (RGB), dus een vorm van (32, 32, 3).

## 📈 Waarom verkennen?
We checken de vorm en kwaliteit van de data.

Het is belangrijk om te zien wat je model gaat leren. Beeldherkenning is een visueel probleem.



## 🧹 Stap 2 – Data omzetten voor klassiek ML‑gebruik
1. **Flattenen**: 32×32×3 → 1×3072 vector.
2. **Schalen**: delen door 255 zodat alle waarden tussen 0–1 liggen.
3. (Optioneel) **Dimensiereductie met PCA** om training te versnellen.

> Bij 50 000 trainingsbeelden × 3072 features kan SVM traag worden. Voor demo‑doeleinden pakken we een **subset van 15 000 samples**.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Flatten en schalen
X_train_flat = X_train.reshape(-1, 32*32*3) / 255.0
X_test_flat  = X_test.reshape(-1, 32*32*3) / 255.0

# Subset voor snelheid
subset = 15000
X_sub, _, y_sub, _ = train_test_split(X_train_flat, y_train, train_size=subset, stratify=y_train, random_state=42)

print('Subset shape:', X_sub.shape)

# PCA naar 100 componenten (versnelt SVM zonder al te veel info te verliezen)
pca = PCA(n_components=100, random_state=42)
X_sub_pca = pca.fit_transform(X_sub)
X_test_pca = pca.transform(X_test_flat)

## 🧠 Stap 3 – Support Vector Machine trainen
We gebruiken een **SVC** met RBF‑kernel. Dit is een krachtig model voor niet‑lineaire patronen. Door PCA is het aantal features nu 100 i.p.v. 3072, waardoor training sneller gaat.

## 🔁 Wat gebeurt er bij max_iter=10?
max_iter=10 betekent: 10 volledige “epochs” over de trainingsdata.

Een epoch = één keer alle trainingsvoorbeelden door het model halen.

Na elke epoch wordt het model bijgesteld op basis van fouten.

## 💬 Waarom herhaalt het zich?
De herhaling (de epochs) zijn nodig omdat:

Het model bij elke ronde bijleert en bijstelt.

Na één ronde (epoch) is het meestal nog niet goed genoeg.

Elke epoch zorgt voor een kleine verbetering, tot het convergentie bereikt of max_iter is bereikt.

In [ ]:
# from sklearn.svm import SVC

# svm = SVC(kernel='rbf', gamma='scale', C=10, random_state=42)
# svm.fit(X_sub_pca, y_sub.ravel())  # ravel → 1D labels
# print('Model getraind!')

from sklearn.linear_model import SGDClassifier

svm = SGDClassifier(loss='hinge', max_iter=10, verbose=1)
svm.fit(X_sub_pca, y_sub.ravel())
print('Model getraind!')

## 🔍 Wat is een SVM?
Een SVM probeert een grensvlak te vinden tussen verschillende klassen in de data.

Met een RBF-kernel kunnen we ook niet-lineaire grenzen trekken (bijvoorbeeld als honden en katten niet perfect gescheiden zijn in een rechte lijn).

C regelt de mate waarin fouten worden bestraft (groter = strenger)

## 🧠 Waarom is dit klassiek ML?
We voeren het model platte rijen van getallen in, geen ruwe beelden

Geen deep learning-lagen, geen automatische feature learning – alles moet vooraf worden klaargemaakt



## ✅ Stap 4 – Evaluatie op de testset

We testen het model op de volledige testset.

We kijken naar **accuracy**, een classificatierapport en een verwarringsmatrix om te zien welke klassen moeilijk zijn.

classification_report() toont:

**Precision**: Hoeveel van de voorspelde katten wáren echt katten?

**Recall**: Hoeveel van alle echte katten heeft het model gevonden?

**F1-score**: Gemiddelde tussen precision en recall

**Accuracy**: Hoeveel is er totaal goed voorspeld?

## 📉 Je mag verwachten:
Accuracy is meestal rond 35–45%. Dat lijkt laag, maar met 10 klassen is random gokken slechts 10%.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = svm.predict(X_test_pca)

acc = accuracy_score(y_test, y_pred)
print(f'Test‑accuracy: {acc:.2f}')

print(classification_report(y_test, y_pred, target_names=class_names))

### 🔎 Verwarringsmatrix visualiseren

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()
plt.xticks(np.arange(10), class_names, rotation=45)
plt.yticks(np.arange(10), class_names)
plt.ylabel('Echte label')
plt.xlabel('Voorspelde label')
plt.tight_layout()
plt.show()

## 🖼️ Bonus – Enkele voorbeelden inspecteren
Bekijk tien willekeurige testbeelden met de voorspelling (groen = correct, rood = fout).

In [ ]:
indices = np.random.choice(len(X_test), 10, replace=False)
plt.figure(figsize=(10,4))
for i, idx in enumerate(indices):
    img = X_test[idx]
    true_label = int(y_test[idx])
    pred_label = int(y_pred[idx])
    plt.subplot(2,5,i+1)
    plt.imshow(img)
    kleur = 'green' if true_label==pred_label else 'red'
    plt.title(class_names[pred_label], color=kleur)
    plt.axis('off')
plt.tight_layout()
plt.show()

## 📊 Reflectie
- **Accuracy** zal lager zijn dan bij een CNN (vaak ~35‑45 %).
- Toch zie je dat **klassieke ML** ook op beelddata toepasbaar is.
- Voor productie‑toepassingen met afbeeldingen is **deep learning** meestal de betere keuze, maar dit experiment toont hoe data‑representatie een groot verschil maakt.

**Vervolgideeën:**
1. Speel met aantal PCA‑componenten en `C`, `gamma` van de SVM.
2. Probeer andere ML‑modellen zoals RandomForest of K‑NN.